In [ ]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm
import os
from scipy import stats
import matplotlib.pyplot as plt
from collections import defaultdict
import crypten
import crypten.nn as cnn
import time
from copy import deepcopy

In [ ]:
def load_dataset(batch_size=128, num_workers=2):
    temp_dataset = datasets.CIFAR10(root='./data', train=True, download=True,
                                     transform=transforms.ToTensor())
    temp_loader = DataLoader(temp_dataset, batch_size=batch_size, num_workers=num_workers)

    channels_sum = t.zeros(3)
    channels_squared_sum = t.zeros(3)
    num_pixels = 0

    for images, _ in temp_loader:
        channels_sum += images.sum(dim=[0, 2, 3])
        channels_squared_sum += (images ** 2).sum(dim=[0, 2, 3])
        num_pixels += images.size(0) * images.size(2) * images.size(3)

    mean = channels_sum / num_pixels
    std = ((channels_squared_sum / num_pixels) - (mean ** 2)) ** 0.5

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean.tolist(), std.tolist())
    ])

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, test_loader

In [ ]:
# TODO: Generalise these, add PlainTextTanh models, ensure the architecture follows SigGuard/MPCDiff closely, Test end to end
class PlainTextCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.activation1 = nn.Sigmoid()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.activation2 = nn.Sigmoid()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.activation3 = nn.Sigmoid()
        self.fc2 = nn.Linear(512, num_classes)

        self.network = nn.Sequential(
            self.conv1,
            self.activation1,
            self.pool1,
            self.conv2,
            self.activation2,
            self.pool2,
            self.flatten,
            self.fc1,
            self.activation3,
            self.fc2
        )

    def forward(self, x):
        return self.network(x)


class PlainTextMLP(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(3072, 512)
        self.activation1 = nn.Sigmoid()
        self.fc2 = nn.Linear(512, 256)
        self.activation2 = nn.Sigmoid()
        self.fc3 = nn.Linear(256, num_classes)

        self.network = nn.Sequential(
            self.flatten,
            self.fc1,
            self.activation1,
            self.fc2,
            self.activation2,
            self.fc3
        )

    def forward(self, x):
        return self.network(x)

class PlainTextLeNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)
        self.activation1 = nn.Sigmoid()
        self.pool1 = nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.activation2 = nn.Sigmoid()
        self.pool2 = nn.AvgPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.activation3 = nn.Sigmoid()
        self.fc2 = nn.Linear(120, 84)
        self.activation4 = nn.Sigmoid()
        self.fc3 = nn.Linear(84, num_classes)

        self.network = nn.Sequential(
            self.conv1,
            self.activation1,
            self.pool1,
            self.conv2,
            self.activation2,
            self.pool2,
            self.flatten,
            self.fc1,
            self.activation3,
            self.fc2,
            self.activation4,
            self.fc3
        )

    def forward(self, x):
        return self.network(x)

In [7]:
def plaintext_train_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc='Training', leave=False)
    for inputs, targets in pbar:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predictions = outputs.argmax(dim=1)
        correct += (predictions == targets).sum().item()
        total += targets.size(0)

        pbar.set_postfix({'loss': running_loss / (pbar.n + 1), 'acc': 100.0 * correct / total})

    avg_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total

    return avg_loss, accuracy


def plaintext_train_model(model, train_loader, num_epochs=10, lr=0.001, device='cuda'):
    model = model.to(device)
    optimizer = t.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    history = {
        'train_loss': [],
        'train_acc': []
    }

    for epoch in range(num_epochs):
        train_loss, train_acc = plaintext_train_epoch(model, train_loader, optimizer, criterion, device)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%')

    return model, history

In [8]:
def train_plaintext_models(epochs=10):
    device = 'cuda' if t.cuda.is_available() else 'cpu'

    train_loader, test_loader = load_dataset(batch_size=128, num_workers=2)

    models = {
        'PlainTextCNN': PlainTextCNN(num_classes=10),
        'PlainTextMLP': PlainTextMLP(num_classes=10),
        'PlainTextLeNet': PlainTextLeNet(num_classes=10)
    }

    os.makedirs('./weights', exist_ok=True)

    for model_name, model in models.items():
        print(f'\nTraining {model_name}...')
        trained_model, history = plaintext_train_model(
            model=model,
            train_loader=train_loader,
            num_epochs=epochs,
            lr=1e-3,
            device=device
        )

        final_weights_path = f'./weights/{model_name}_final.pt'
        t.save(trained_model.state_dict(), final_weights_path)
        print(f'Final weights saved: {final_weights_path}')


train_plaintext_models(40)

100.0%



Training PlainTextCNN...


KeyboardInterrupt: 

In [9]:
def evaluate_accuracy_loss(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with t.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            predictions = outputs.argmax(dim=1)
            correct += (predictions == targets).sum().item()
            total += targets.size(0)

    avg_loss = running_loss / len(test_loader)
    accuracy = 100.0 * correct / total

    return avg_loss, accuracy

In [ ]:
def load_model_from_weights(model_class, weights_path, num_classes=10, device='cuda'):
    model = model_class(num_classes=num_classes)
    model.load_state_dict(t.load(weights_path, map_location=device, weights_only=True))
    model = model.to(device)
    print(f'Loaded weights from: {weights_path}')
    return model


def continue_training(model, train_loader, num_epochs=10, lr=0.001, device='cuda',
                      start_epoch=0, save_path=None):

    model = model.to(device)
    optimizer = t.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    history = {
        'train_loss': [],
        'train_acc': []
    }

    for epoch in range(num_epochs):
        train_loss, train_acc = plaintext_train_epoch(model, train_loader, optimizer, criterion, device)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        display_epoch = start_epoch + epoch + 1
        print(f'Epoch {display_epoch} - Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%')

    if save_path is not None:
        t.save(model.state_dict(), save_path)
        print(f'Weights saved: {save_path}')

    return model, history


def load_and_continue_training(model_class, weights_path, train_loader, num_epochs=10,
                                lr=0.001, device='cuda', start_epoch=0, save_path=None):

    model = load_model_from_weights(model_class, weights_path, device=device)
    model, history = continue_training(
        model=model,
        train_loader=train_loader,
        num_epochs=num_epochs,
        lr=lr,
        device=device,
        start_epoch=start_epoch,
        save_path=save_path
    )
    return model, history

FileNotFoundError: [Errno 2] No such file or directory: './weights/PlainTextLeNet_final.pt'

In [ ]:
# CrypTen models require manual flattening

class MpcFlatten(cnn.Module):
    def forward(self, x):
        return x.flatten(start_dim=1)

class MpcCNN(cnn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = cnn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.activation1 = cnn.Sigmoid() 
        # Note that MaxPool2d is significantly more expensive in MPC
        self.pool1 = cnn.MaxPool2d(2, 2)
        self.conv2 = cnn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.activation2 = cnn.Sigmoid()
        self.pool2 = cnn.MaxPool2d(2, 2)
        
        self.flatten = MpcFlatten() # replace nn.Flatten
        
        self.fc1 = cnn.Linear(64 * 8 * 8, 512)
        self.activation3 = cnn.Sigmoid()
        self.fc2 = cnn.Linear(512, num_classes)

        self.network = cnn.Sequential(
            self.conv1,
            self.activation1,
            self.pool1,
            self.conv2,
            self.activation2,
            self.pool2,
            self.flatten,
            self.fc1,
            self.activation3,
            self.fc2
        )

    def forward(self, x):
        return self.network(x)


class MpcMLP(cnn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.flatten = MpcFlatten() 
        self.fc1 = cnn.Linear(3072, 512)
        self.activation1 = cnn.Sigmoid()
        self.fc2 = cnn.Linear(512, 256)
        self.activation2 = cnn.Sigmoid()
        self.fc3 = cnn.Linear(256, num_classes)

        self.network = cnn.Sequential(
            self.flatten,
            self.fc1,
            self.activation1,
            self.fc2,
            self.activation2,
            self.fc3
        )

    def forward(self, x):
        return self.network(x)

class MpcLeNet(cnn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = cnn.Conv2d(3, 6, kernel_size=5, padding=2)
        self.activation1 = cnn.Sigmoid()
        self.pool1 = cnn.AvgPool2d(2, 2) # AvgPool is efficient in MPC
        
        self.conv2 = cnn.Conv2d(6, 16, kernel_size=5)
        self.activation2 = cnn.Sigmoid()
        self.pool2 = cnn.AvgPool2d(2, 2)
        
        self.flatten = MpcFlatten() 
        self.fc1 = cnn.Linear(16 * 6 * 6, 120)
        self.activation3 = cnn.Sigmoid()
        self.fc2 = cnn.Linear(120, 84)
        self.activation4 = cnn.Sigmoid()
        self.fc3 = cnn.Linear(84, num_classes)

        self.network = cnn.Sequential(
            self.conv1,
            self.activation1,
            self.pool1,
            self.conv2,
            self.activation2,
            self.pool2,
            self.flatten,
            self.fc1,
            self.activation3,
            self.fc2,
            self.activation4,
            self.fc3
        )

    def forward(self, x):
        return self.network(x)

In [ ]:

# TODO: Just generalize the construction of these models so we can change the activation functions easily
class MpcCNNTanh(cnn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = cnn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.activation1 = cnn.Tanh()  
        self.pool1 = cnn.MaxPool2d(2, 2)
        
        self.conv2 = cnn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.activation2 = cnn.Tanh()  
        self.pool2 = cnn.MaxPool2d(2, 2)
        
        self.flatten = MpcFlatten() 
        
        self.fc1 = cnn.Linear(64 * 8 * 8, 512)
        self.activation3 = cnn.Tanh()  
        self.fc2 = cnn.Linear(512, num_classes)

        self.network = cnn.Sequential(
            self.conv1,
            self.activation1,
            self.pool1,
            self.conv2,
            self.activation2,
            self.pool2,
            self.flatten,
            self.fc1,
            self.activation3,
            self.fc2
        )

    def forward(self, x):
        return self.network(x)

class MpcMLPTanh(cnn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.flatten = MpcFlatten()
        self.fc1 = cnn.Linear(3072, 512)
        self.activation1 = cnn.Tanh()  
        self.fc2 = cnn.Linear(512, 256)
        self.activation2 = cnn.Tanh()  
        self.fc3 = cnn.Linear(256, num_classes)

        self.network = cnn.Sequential(
            self.flatten,
            self.fc1,
            self.activation1,
            self.fc2,
            self.activation2,
            self.fc3
        )

    def forward(self, x):
        return self.network(x)


class MpcLeNetTanh(cnn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = cnn.Conv2d(3, 6, kernel_size=5, padding=2)
        self.activation1 = cnn.Tanh()
        self.pool1 = cnn.AvgPool2d(2, 2)
        
        self.conv2 = cnn.Conv2d(6, 16, kernel_size=5)
        self.activation2 = cnn.Tanh()  
        self.pool2 = cnn.AvgPool2d(2, 2)
        
        self.flatten = MpcFlatten()
        
        self.fc1 = cnn.Linear(16 * 6 * 6, 120)
        self.activation3 = cnn.Tanh()  
        self.fc2 = cnn.Linear(120, 84)
        self.activation4 = cnn.Tanh()  
        self.fc3 = cnn.Linear(84, num_classes)

        self.network = cnn.Sequential(
            self.conv1,
            self.activation1,
            self.pool1,
            self.conv2,
            self.activation2,
            self.pool2,
            self.flatten,
            self.fc1,
            self.activation3,
            self.fc2,
            self.activation4,
            self.fc3
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
def mpc_train_epoch(model, train_loader, optimizer, criterion, device, num_classes=10):

    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        x_enc = crypten.cryptensor(inputs) # Encrypt inputs      
        y_one_hot = F.one_hot(targets, num_classes=num_classes).float() # Prepare Targets (One-Hot Encoding is required for MPC Loss) (TODO: Look into this)
        y_enc = crypten.cryptensor(y_one_hot)
        optimizer.zero_grad()
        
        output_enc = model(x_enc) # Encrypted Forward Pass
        loss_enc = criterion(output_enc, y_enc)
        loss_enc.backward() # Encrypted Backward Pass 
        optimizer.step()   

        loss_val = loss_enc.get_plain_text().item() # Decrypt for metrics calculation 
        running_loss += loss_val
        
        # Decrypt outputs to calculate accuracy 
        output_plain = output_enc.get_plain_text()
        predictions = output_plain.argmax(dim=1)
        correct += (predictions == targets).sum().item()
        total += targets.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total
    
    return avg_loss, accuracy

def mpc_train_model(model, train_loader, num_epochs=10, lr=0.001, device='cpu'):

    if not crypten.is_initialized():
        crypten.init()
    
    model.encrypt() # Encrypt the model (Weights become CrypTensors)
    model.train()

    optimizer = crypten.optim.Adam(model.parameters(), lr=lr) # Use CrypTen implementation of Adam
    criterion = cnn.CrossEntropyLoss() # Crypten implementation of CrossEntropyLoss

    history = {
        'train_loss': [],
        'train_acc': []
    }

    print(f"Starting MPC Training for {num_epochs} epochs...")
    start_time = time.time()

    for epoch in range(num_epochs):
        train_loss, train_acc = mpc_train_epoch(
            model, 
            train_loader, 
            optimizer, 
            criterion, 
            device
        )

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        
        elapsed = time.time() - start_time
        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} | Acc: {train_acc:.2f}% | Time: {elapsed:.0f}s')

    return model, history

def train_mpc_models(epochs=10):

    train_loader, test_loader = load_dataset(batch_size=32, num_workers=2) # Reduced batch size for MPC memory safety

    models = {
        'MpcCNN': MpcCNN(num_classes=10),
        'MpcMLP': MpcMLP(num_classes=10),
        'MpcLeNet': MpcLeNet(num_classes=10)
    }

    os.makedirs('./weights_mpc', exist_ok=True)

    for model_name, model in models.items():
        print(f'\nTraining {model_name} in MPC...')
        
        trained_model, history = mpc_train_model(
            model=model,
            train_loader=train_loader,
            num_epochs=epochs,
            lr=1e-3
        )
        
        # Save encrypted model state
        final_weights_path = f'./weights_mpc/{model_name}_encrypted.pt'
        crypten.save(trained_model.state_dict(), final_weights_path)
        print(f'Encrypted weights saved: {final_weights_path}')

train_mpc_models(epochs=1) 

In [ ]:
def train_shadow_models(num_shadows, model_class, full_dataset, num_epochs=10, device='cuda'):
    """
    Trains a set of shadow models on random subsets of the data. # TODO: Fix a portion of CIFAR10/Whatever dataset to train shadow models on? 
    Returns:
        shadow_models (list): List of trained shadow models.
        shadow_data (list): List of tuples (train_indices, test_indices) used for each model.
    """
    shadow_models = []
    shadow_data_indices = []
    dataset_size = len(full_dataset)
    split_size = dataset_size // 2  # 50% in, 50% out (Check Shokri Figures)
    
    print(f"Training {num_shadows} shadow models...")
    
    for i in range(num_shadows):
        indices = np.random.permutation(dataset_size)  # Create random split for this shadow model
        train_indices = indices[:split_size]
        test_indices = indices[split_size:]        
        train_subset = Subset(full_dataset, train_indices)

        train_loader = DataLoader(train_subset, batch_size=128, shuffle=True, num_workers=0)
        shadow_model = model_class(num_classes=10).to(device) # Initialize and a train shadow model
        
        print(f"Shadow Model {i+1}/{num_shadows}...")
        plaintext_train_model(
            shadow_model, 
            train_loader, 
            num_epochs=num_epochs, 
            lr=1e-3, 
            device=device
        )
        shadow_models.append(shadow_model)
        shadow_data_indices.append((train_indices, test_indices))
        
    return shadow_models, shadow_data_indices

In [ ]:
class AttackNet(nn.Module):
    def __init__(self, input_dim=10):
        super().__init__()
        # Input is the target model's logit vector (size 10 for CIFAR-10) 
        self.fc1 = nn.Linear(input_dim, 64)
        self.activation1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.activation2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1) # 0 Non-Member, 1 Member
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.activation1(self.fc1(x))
        x = self.activation2(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

def prepare_attack_dataset(shadow_models, shadow_indices, full_dataset, device='cuda'):
 #   Generate (logit, membership_label) pairs from shadow models.
    X_attack = []
    y_attack = []
    
    full_loader = DataLoader(full_dataset, batch_size=128, shuffle=False, num_workers=0)
    
    with t.no_grad():
        for i, model in enumerate(shadow_models):
            model.eval()
            train_idx, test_idx = shadow_indices[i]
            train_set = set(train_idx)
            
            # Get predictions for the whole dataset
            all_preds = []
            for inputs, _ in full_loader:
                inputs = inputs.to(device)
                outputs = model(inputs)
                preds = F.softmax(outputs, dim=1)# Convert logits to probabilities
                all_preds.append(preds.cpu())

            all_preds = t.cat(all_preds)
            
            # Label '1' for members (in train_set), '0' for non-members

            for idx in range(len(full_dataset)):
                pred_vector = all_preds[idx]
                label = 1.0 if idx in train_set else 0.0
                
                X_attack.append(pred_vector)
                y_attack.append(label)
                
    X_attack = t.stack(X_attack)
    y_attack = t.tensor(y_attack).unsqueeze(1) # Shape [N, 1]
    
    return X_attack, y_attack

def train_attack_model(X_attack, y_attack, epochs=20, device='cuda'):

    attack_model = AttackNet().to(device)
    optimizer = t.optim.Adam(attack_model.parameters(), lr=0.001)
    criterion = nn.BCELoss() # Binary Cross Entropy Loss
    
    dataset = t.utils.data.TensorDataset(X_attack, y_attack)
    loader = DataLoader(dataset, batch_size=64, shuffle=True)
    
    print("Training Attack Model...")
    for epoch in range(epochs):
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = attack_model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
    return attack_model

In [ ]:
def evaluate_mia_attack(target_model, attack_model, train_loader, test_loader, device, is_mpc=False):
    """
    Runs the trained attack model on the target model's members (train_loader) 
    and non-members (test_loader) to calculate attack accuracy.
    TODO: Construct a mixed dataset to test on
    """

    target_model.eval()
    attack_model.eval()
    
    # Helper to get predictions from the target

    def get_target_preds(loader, is_member):
        preds = []
        labels = []
        
        for inputs, _ in loader:

            if is_mpc:

                # MPC Path: Encrypt -> Forward -> Decrypt
                x_enc = crypten.cryptensor(inputs)
                output_enc = target_model(x_enc)
                output_plain = output_enc.get_plain_text()
                
                # Apply softmax on plaintext for the attack features
                # (Attacker sees probabilities)
                batch_preds = F.softmax(output_plain, dim = 1)

            else:
                # Plaintext Path
                inputs = inputs.to(device)
                with t.no_grad():
                    outputs = target_model(inputs)
                    batch_preds = softmax(outputs)
            
            preds.append(batch_preds.cpu())
            # 1.0 for Members, 0.0 for Non-Members
            labels.extend([1.0 if is_member else 0.0] * inputs.size(0))
        return t.cat(preds), t.tensor(labels).unsqueeze(1)

    print(f"Collecting predictions from {'MPC' if is_mpc else 'Plaintext'} Target...")
    
    # Get predictions on Member data (Train set)
    member_preds, member_labels = get_target_preds(train_loader, is_member=True)
    
    # Get predictions on Non-Member data (Test set)
    non_member_preds, non_member_labels = get_target_preds(test_loader, is_member=False)
    
    all_preds = t.cat([member_preds, non_member_preds])
    all_labels = t.cat([member_labels, non_member_labels])
    
    with t.no_grad():
        attack_probs = attack_model(all_preds.to(device))
        attack_preds = (attack_probs > 0.5).float().cpu()
        
    correct = (attack_preds == all_labels).sum().item()
    total = all_labels.size(0)
    accuracy = 100.0 * correct / total
    
    tp = ((attack_preds == 1) & (all_labels == 1)).sum().item()
    fp = ((attack_preds == 1) & (all_labels == 0)).sum().item()
    fn = ((attack_preds == 0) & (all_labels == 1)).sum().item()
    
    precision = 100.0 * tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = 100.0 * tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print(f"MIA Accuracy: {accuracy:.2f}%")
    print(f"MIA Precision (Member): {precision:.2f}%")
    print(f"MIA Recall (Member): {recall:.2f}%")
    
    return accuracy, precision, recall